# MSS: Get WT scores for a bunch of models

In [1]:
import sys

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.stats import spearmanr

pd.set_option("display.max_columns", 100)
sys.path.append("..")

In [2]:
# Inputs:
dms_reference_file = "../reference_files/DMS_substitutions.csv"
scores_file = "/Users/admin/Documents/Work/2022 Harvard/ProteinGym/benchmarks/DMS_zero_shot/substitutions/Spearman/DMS_substitutions_Spearman_DMS_level.csv"

In [3]:
# Read the reference file
df_ref = pd.read_csv(dms_reference_file)
df_ref

,DMS_id,DMS_filename,UniProt_ID,taxon,source_organism,target_seq,seq_len,includes_multiple_mutants,DMS_total_number_mutants,DMS_number_single_mutants,DMS_number_multiple_mutants,DMS_binarization_cutoff,DMS_binarization_method,first_author,title,year,jo,region_mutated,molecule_name,selection_assay,selection_type,MSA_filename,MSA_start,MSA_end,MSA_len,MSA_bitscore,MSA_theta,MSA_num_seqs,MSA_perc_cov,MSA_num_cov,MSA_N_eff,MSA_Neff_L,MSA_Neff_L_category,MSA_num_significant,MSA_num_significant_L,raw_DMS_filename,raw_DMS_phenotype_name,raw_DMS_directionality,raw_DMS_mutant_column,weight_file_name,pdb_file,pdb_range,ProteinGym_version,raw_mut_offset,coarse_selection_type
0,A0A140D2T1_ZIKV_Sourisseau_2019,A0A140D2T1_ZIKV_Sourisseau_2019.csv,A0A140D2T1_ZIKV,Virus,Zika virus,MKNPKKKSGGFRIVNMLKRGVARVNPLGGLKRLPAGLLLGHGPIRM...,3423,False,9576,9576,0,0.043249,median,Sourisseau,Deep Mutational Scanning Comprehensively Maps ...,2019,10.1128/JVI.01291-19,291-794,Zika virus env,Viral replication,Growth,A0A140D2T1_ZIKV_theta0.99_281-804_11-26-2021_b...,281,804,524,0.2,0.01,16501,0.948,497.0,1357.9,2.732193,medium,329.0,0.661972,A0A140D2T1_ZIKV_Sourisseau_growth_2019.csv,effect,1,mutant,A0A140D2T1_ZIKV_theta_0.01.npy,A0A140D2T1_ZIKV.pdb,291-794,0.1,NaN,OrganismalFitness
1,A0A192B1T2_9HIV1_Haddox_2018,A0A192B1T2_9HIV1_Haddox_2018.csv,A0A192B1T2_9HIV1,Virus,HIV,MRVKGIQMNSQHLLRWGIMILGMIMICSVAGNLWVTVYYGVPVWKD...,852,False,12577,12577,0,-2.200000,manual,Haddox,Mapping mutational effects along the evolution...,2018,10.7554/eLife.34420,30-691,HIV env (BF520),Viral replication,Growth,A0A192B1T2_9HIV1_theta0.99_full_11-26-2021_b09...,1,852,852,0.9,0.01,74854,0.986,840.0,36319.9,43.237976,medium,2382.0,2.835714,A0A192B1T2_9HIV1_Haddox_2018.csv,fitness,1,mutant,A0A192B1T2_9HIV1_theta_0.01.npy,A0A192B1T2_9HIV1.pdb,1-852,0.1,NaN,OrganismalFitness
2,A0A1I9GEU1_NEIME_Kennouche_2019,A0A1I9GEU1_NEIME_Kennouche_2019.csv,A0A1I9GEU1_NEIME,Prokaryote,Neisseria meningitidis,FTLIELMIVIAIVGILAAVALPAYQDYTARAQVSEAILLAEGQKSA...,161,False,922,922,0,0.141000,median,Kennouche,Deep mutational scanning of the Neisseria meni...,2019,10.15252/embj.2019102145,1-161,pilin (PilE),"piliation (20D9 anti-pilus monoclonal Ab), agg...",NaN,A0A1I9GEU1_NEIME_full_11-26-2021_b08.a2m,1,161,161,0.8,0.20,5553,0.857,138.0,2183.6,15.823188,medium,72.0,0.521739,A0A1I9GEU1_NEIME_Kennouche_2019.csv,piliation_log2_ratio,1,mutants,A0A1I9GEU1_NEIME_theta_0.2.npy,A0A1I9GEU1_NEIME.pdb,1-161,0.1,NaN,Activity
3,A0A247D711_LISMN_Stadelmann_2021,A0A247D711_LISMN_Stadelmann_2021.csv,A0A247D711_LISMN,Eukaryote,Listeria monocytogenes,MNINDLIREIKNKDYTVKLSGTDSNSITQLIIRVNNDGNEYVISES...,87,False,1653,1653,0,-0.015563,median,Stadelmann,A deep mutational scanning platform to charact...,2021,10.1101/2021.08.21.457204,1-87,Anti-CRISPR protein AcrIIA4,activity against SpyCas9 inducing an RFP reporter,Flow cytometry,A0A247D711_LISMN_full_b0.3.a2m,1,87,87,0.2,0.20,1316890,1.000,87.0,188739.9,2169.424138,High,209.0,2.402299,A0A247D711_LISMN_Stadelmann_2021.csv,mean_prediction,1,mutant,A0A247D711_LISMN_b03_theta_0.2.npy,A0A247D711_LISMN.pdb,1-87,1.0,NaN,Activity
4,A0A2Z5U3Z0_9INFA_Doud_2016,A0A2Z5U3Z0_9INFA_Doud_2016.csv,A0A2Z5U3Z0_9INFA,Virus,influenza H1N1,MKAKLLVLLYAFVATDADTICIGYHANNSTDTVDTILEKNVAVTHS...,565,False,10715,10715,0,-2.239943,median,Doud,Accurate Measurement of the Effects of All Ami...,2016,10.3390/v8060155,2-565,Influenza hemagglutinin,viral replication,Growth,A0A2Z5U3Z0_9INFA_theta0.99_full_11-26-2021_b09...,1,565,565,0.9,0.01,57581,0.968,547.0,9809.4,17.933090,medium,925.0,1.691042,A0A2Z5U3Z0_9INFA_Doud_2016.csv,transformed_pref,1,mutant,A0A2Z5U3Z0_9INFA_theta_0.01.npy,A0A2Z5U3Z0_9INFA.pdb,1-565,0.1,NaN,OrganismalFitness
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,VKOR1_HUMAN_Chiasson_2020_activity,VKOR1_HUMAN_Chiasson_2020_activity.csv,VKOR1_HUMAN,Human,Homo sap

In [4]:
example_mapping = pd.DataFrame({"DMS_id": ["all_wt"], "DMS_filename": ["all_wt.csv"]})  # TODO later this should just point to a CSV / fasta to score all sequences
# example_mapping.to_csv("all_wt_mapping.csv", index=False)

In [5]:
example_csv = df_ref[["target_seq"]].drop_duplicates().reset_index(drop=True).rename(columns={"target_seq": "mutated_sequence"})
example_csv["DMS_score"] = 0
# example_csv.to_csv("all_wt.csv", index=False)
example_csv

,mutated_sequence,DMS_score
0,MKNPKKKSGGFRIVNMLKRGVARVNPLGGLKRLPAGLLLGHGPIRM...,0
1,MRVKGIQMNSQHLLRWGIMILGMIMICSVAGNLWVTVYYGVPVWKD...,0
2,FTLIELMIVIAIVGILAAVALPAYQDYTARAQVSEAILLAEGQKSA...,0
3,MNINDLIREIKNKDYTVKLSGTDSNSITQLIIRVNNDGNEYVISES...,0
4,MKAKLLVLLYAFVATDADTICIGYHANNSTDTVDTILEKNVAVTHS...,0
...,...,...
182,MGSTWGSPGWVRLALCLTGLVLSLYALHVKAARARDRDYRALCDVG...,0
183,SLSVDNKKFWATVESSEHSFEVPIYAETLDEALELAEWQYVPAGFE...,0
184,PREAYIVTIEKGKPGQTVTWYQLRADHPKPDSLISEHPTAQEAMDA...,0
185,MDPGQQPPPQPAPQGQGQPPSQPPQGQGPPSGPGQPAPAATQAAPQ...,0


In [6]:
# Make per-sequence files
example_mapping_per_sequence = df_ref.drop_duplicates(subset="target_seq")[["DMS_id", "DMS_filename"]]
# example_mapping_per_sequence.to_csv("per_wt_mapping.csv", index=False)
for i, row in df_ref.drop_duplicates(subset="target_seq").iterrows():
    tmp_df = pd.DataFrame([row["target_seq"]], columns=["mutated_sequence"]).rename(columns={"target_seq": "mutated_sequence"})
    tmp_df["DMS_score"] = 0
    # tmp_df.to_csv("./per_wt_seq/" + row["DMS_filename"], index=False)

## Example 1: Get ProGen scores (start small?)

In [7]:
# Takes 9 seconds for 2 seqs, 217 sequences would take 15min

index = 0
command = f"../proteingym/baselines/progen2/compute_fitness.py \
            --Progen2_model_name_or_path ../checkpoints/progen2-base \
            --DMS_reference_file_path all_wt_mapping.csv \
            --DMS_data_folder . \
            --DMS_index {index} \
            --output_scores_folder ./model_scores_out/ \
            --indel_mode \
            --device mps \
            --test \
            --raw_score"
# command = f"../proteingym/baselines/progen2/likelihood.py --model_ckpt ../checkpoints/progen2-base/  \
#     --context {example_csv.iloc[index]['mutated_sequence']}  \
#     --device mps"
! python {command}

Maximum context length: 2048
Computing scores for: all_wt with Progen2: ../checkpoints/progen2-base
Model: progen2-base
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.17it/s]
[-409.02035522] -1.9 [407.12035522]
Traceback (most recent call last):
  File "/Users/admin/Documents/Work/2022 Harvard/ProteinGym/notebooks/../proteingym/baselines/progen2/compute_fitness.py", line 185, in <module>
    main()
  File "/Users/admin/Documents/Work/2022 Harvard/ProteinGym/notebooks/../proteingym/baselines/progen2/compute_fitness.py", line 172, in main
    assert abs(model_scores - checkpoint_x_ll[model_size][1]) < 0.1
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError


## ESM pseudolikelihoods

In [8]:
# Stitch together output files into all_wt.csv
dirs = ["esm2_t30_150M_UR50D", "esm1v_t33_650M_UR90S_1"]
import os
for dir in dirs:
    dfs = []
    dir_path = f"./model_scores_out/{dir}/"
    for filename in sorted(os.listdir(dir_path)):
        if filename == "all_wt.csv":
            continue
        dms_id = filename.split(".csv")[0]
        tmp = pd.read_csv(dir_path+filename)
        tmp["DMS_id"] = dms_id
        dfs.append(tmp)
    tmp_full = pd.concat(dfs)
    model_col = dir
    tmp_full[model_col] = tmp_full[model_col].replace(0, np.nan)
    tmp_full = tmp_full.drop(columns=[c for c in tmp_full if "Ensemble" in c])
    tmp_full = tmp_full.drop(columns=["DMS_score"])
    tmp_full.to_csv(f"{dir_path}/all_wt.csv", index=False)

df_tmp = pd.read_csv("./model_scores_out/esm1v_t33_650M_UR90S_1_wt_pseudo_ppl/all_wt.csv")
df_tmp

,mutated_sequence,DMS_score,esm1v_t33_650M_UR90S_1,Ensemble_ESM1v
0,MKNPKKKSGGFRIVNMLKRGVARVNPLGGLKRLPAGLLLGHGPIRM...,0,-12941.488922,-12941.488922
1,MRVKGIQMNSQHLLRWGIMILGMIMICSVAGNLWVTVYYGVPVWKD...,0,-7479.261695,-7479.261695
2,FTLIELMIVIAIVGILAAVALPAYQDYTARAQVSEAILLAEGQKSA...,0,-1550.662315,-1550.662315
3,MNINDLIREIKNKDYTVKLSGTDSNSITQLIIRVNNDGNEYVISES...,0,-354.171648,-354.171648
4,MKAKLLVLLYAFVATDADTICIGYHANNSTDTVDTILEKNVAVTHS...,0,-5073.500586,-5073.500586
...,...,...,...,...
182,MGSTWGSPGWVRLALCLTGLVLSLYALHVKAARARDRDYRALCDVG...,0,-1218.057225,-1218.057225
183,SLSVDNKKFWATVESSEHSFEVPIYAETLDEALELAEWQYVPAGFE...,0,-226.470604,-226.470604
184,PREAYIVTIEKGKPGQTVTWYQLRADHPKPDSLISEHPTAQEAMDA...,0,-220.723548,-220.723548
185,MDPGQQPPPQPAPQGQGQPPSQPPQGQGPPSGPGQPAPAATQAAPQ...,0,-4214.837010,-4214.837010


In [9]:
# tmp compute using ESM2 endpoint
# token = HUGGINGFACE_TOKEN

# import requests

# API_URL = "https://api-inference.huggingface.co/models/facebook/esm2_t6_8M_UR50D"
# headers = {"Authorization": f"Bearer {token}"}

# def query(payload):
# 	response = requests.post(API_URL, headers=headers, json=payload)
# 	return response.json()
	
# output = query({
# 	"inputs": "MQIFVKTLTGKTITLEVEPS<mask>MIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG",
# })

In [46]:
# 144s per sequence, would take 8 hours...
# 2 positions per second
# esm1v_t33_650M_UR90S_1, but now using much smaller one

# Using one file for all the sequences
# Using wt-marginals pseudo-ppl (experimental)
# command = f"../proteingym/baselines/esm/compute_fitness.py \
#     --model-location esm1v_t33_650M_UR90S_1 \
#     --model_type ESM1v \
#     --dms_index 0 \
#     --dms_mapping all_wt_mapping.csv \
#     --dms-input . \
#     --dms-output ./model_scores_out/esm1v_t33_650M_UR90S_1_wt_pseudo_ppl/ \
#     --scoring-strategy wt-pseudo-ppl \
#     --scoring-window optimal \
#     --device mps"  # Should try use mps
# ! python {command}

# Use per-sequence mapping file
# for i in range(0, 188):
#     command = f"../proteingym/baselines/esm/compute_fitness.py \
#         --model-location esm1v_t33_650M_UR90S_1 \
#         --model_type ESM1v \
#         --dms_index {i} \
#         --dms_mapping per_wt_mapping.csv \
#         --dms-input ./per_wt_seq/ \
#         --dms-output ./model_scores_out/esm1v_t33_650M_UR90S_1/ \
#         --scoring-strategy pseudo-ppl \
#         --scoring-window optimal \
#         --device mps"  # Should try use mps
#     ! python {command}

# ! echo {1..2} | xargs -n1 -I{} -P 4 python ../proteingym/baselines/esm/compute_fitness.py \
#     --model-location esm2_t30_150M_UR50D \
#     --model_type ESM2 \
#     --dms_index {} \
#     --dms_mapping per_wt_mapping.csv \
#     --dms-input ./per_wt_seq/ \
#     --dms-output ./model_scores_out/esm2_t30_150M_UR50D/ \
#     --scoring-strategy pseudo-ppl \
#     --scoring-window optimal \
#     --device mps

Arguments: Namespace(model_type=['ESM1v'], model_location=['esm1v_t33_650M_UR90S_1'], sequence=None, dms_input=PosixPath('.'), dms_index=0, dms_mapping='all_wt_mapping.csv', mutation_col='mutant', dms_output=PosixPath('model_scores_out/esm1v_t33_650M_UR90S_1_wt_pseudo_ppl'), offset_idx=1, scoring_strategy='wt-pseudo-ppl', msa_path=None, msa_sampling_strategy='sequence-reweighting', msa_samples=400, msa_weights_folder=None, seeds=1, filter_msa=False, hhfilter_min_cov=75, hhfilter_max_seq_id=90, hhfilter_min_seq_id=0, path_to_hhfilter='/n/groups/marks/software/hhsuite/hhsuite-3.3.0', scoring_window='optimal', overwrite_prior_scores=False, device='mps', nogpu=False)
Compute scores for DMS: all_wt
Note: pseudo-ppl not normalizing by WT
tmp df head:                                      mutated_sequence  DMS_score
0  MKNPKKKSGGFRIVNMLKRGVARVNPLGGLKRLPAGLLLGHGPIRM...          0
1  MRVKGIQMNSQHLLRWGIMILGMIMICSVAGNLWVTVYYGVPVWKD...          0
2  FTLIELMIVIAIVGILAAVALPAYQDYTARAQVSEAILLAEGQKSA...

## Test ESM pseudolikelihood directionality

In [ ]:
# Choose a DMS with really high ESM scores


## MSAT

In [10]:
# Using wt-marginals pseudo-ppl (experimental)
# Use per-sequence mapping file
for i in range(0, 188):
    command = f"../proteingym/baselines/esm/compute_fitness.py \
        --model-location esm_msa1b_t12_100M_UR50S \
        --model_type MSA_transformer \
        --dms_index {i} \
        --dms_mapping ../reference_files/DMS_substitutions.csv \
        --dms-input ./per_wt_seq/ \
        --dms-output ./model_scores_out/esm_msa1b_t12_100M_UR50S_wt_pseudo_ppl/ \
        --scoring-strategy wt-pseudo-ppl \
        --scoring-window overlapping \
        --msa-path ../data/DMS_msa_files/ \
        --msa-weights-folder ../data/DMS_msa_weights/ \
        --msa-samples 200 \
        --device mps"  # Should try use mps
    ! python {command}

Arguments: Namespace(model_type=['MSA_transformer'], model_location=['esm_msa1b_t12_100M_UR50S'], sequence=None, dms_input=PosixPath('per_wt_seq'), dms_index=0, dms_mapping='../reference_files/DMS_substitutions.csv', mutation_col='mutant', dms_output=PosixPath('model_scores_out/esm_msa1b_t12_100M_UR50S_wt_pseudo_ppl'), offset_idx=1, scoring_strategy='wt-pseudo-ppl', msa_path=PosixPath('../data/DMS_msa_files'), msa_sampling_strategy='sequence-reweighting', msa_samples=200, msa_weights_folder='../data/DMS_msa_weights/', seeds=[1], filter_msa=False, hhfilter_min_cov=75, hhfilter_max_seq_id=90, hhfilter_min_seq_id=0, path_to_hhfilter='/n/groups/marks/software/hhsuite/hhsuite-3.3.0', scoring_window='overlapping', overwrite_prior_scores=False, device='mps', nogpu=False)
Compute scores for DMS: A0A140D2T1_ZIKV_Sourisseau_2019
Note: pseudo-ppl not normalizing by WT
Chopping sequence by start 281 and end 804
tmp df head:                                      mutated_sequence  DMS_score
0  MKNPKK

## ClinVar genes ranking

In [11]:
# import pandas as pd
# df_clinvar = pd.read_csv("../reference_files/clinical_substitutions.csv")
# mock_df = df_clinvar.copy()
# mock_df["mutated_sequence"] = mock_df["target_seq"]  # Just renaming mutant column basically
# mock_df.to_csv("all_clinvar.csv", index=False)

index = 0
command = f"../proteingym/baselines/progen2/compute_fitness.py \
            --Progen2_model_name_or_path ../checkpoints/progen2-base \
            --DMS_reference_file_path all_clinvar_mapping.csv \
            --DMS_data_folder . \
            --DMS_index {index} \
            --output_scores_folder ./model_scores_out/clinvar/progen2-base/ \
            --indel_mode \
            --device mps \
            --test \
            --raw_score"
# command = f"../proteingym/baselines/progen2/likelihood.py --model_ckpt ../checkpoints/progen2-base/  \
#     --context {example_csv.iloc[index]['mutated_sequence']}  \
#     --device mps"
! python {command}  # Max sequence length is 2302, can actually slice these out if necessary

Maximum context length: 2048
Computing scores for: all_clinvar with Progen2: ../checkpoints/progen2-base
Model: progen2-base
100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.14s/it]
[-1.81786825] -1.9 [0.08213175]
Test: Outputs correct
 20%|███████▏                           | 516/2525 [9:55:11<38:37:20, 69.21s/it]
Traceback (most recent call last):
  File "/Users/admin/Documents/Work/2022 Harvard/ProteinGym/notebooks/../proteingym/baselines/progen2/compute_fitness.py", line 186, in <module>
    main()
  File "/Users/admin/Documents/Work/2022 Harvard/ProteinGym/notebooks/../proteingym/baselines/progen2/compute_fitness.py", line 176, in main
    model_scores = calc_fitness(model=model, prots=np.array(DMS_data['mutated_sequence']), model_context_len=int(config['n_positions']), tokenizer=tokenizer, fp16=args.fp16, device=device)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

### Later

In [ ]:
# Start with ESM, ProGen for example (easiest two?) and get scores for all WT in the substitutions doc
# - use indels scripts, maybe add scripts for unnormalized scores?

## Related: Recalculate Spearmans for only singles (need O2 access)

In [ ]:
# TODOs:
# - Recalculate Spearmans just for singles to ignore "flukes": inflated performance by just separating multiples from singles
# Find a way to extend this to alignments
# - Difficult: Rescore all Rocklin assays in their sequence context (to get full sequence likelihoods)
# - - Although this is already showing a fine correlation just by looking at the tested constructs 
#       -> and plus these have high spearmans anyway so it's not as if the model was broken here
# 